In [1]:

library(glue)

In [5]:

meta <- readRDS("/Users/shayecarver/condPCA/Light_Hvratin/figures/cnmf_usages/meta_QC_GSE102827_MATRIX_20k_vargene.rds")
# obtain time metadata and convert to time continuum
meta$time = as.numeric(substr((meta$stim),1,1))


In [8]:
# Create an empty data frame with the specified column names
column_names <- c("method","r.adj","dim","vargenes","Method","CT")
final_df <- data.frame(matrix(nrow = 0, ncol = length(column_names)))
colnames(final_df) <- column_names

compute_rsq <- function(int_ex_OR_all_cell_types, vargenes,dim, ct){
    # read in embeddings
    embeddings <- read.table(glue("/Users/shayecarver/condPCA/Light_Hvratin/figures/cnmf_usages/cNMF_cells_all_seed_1_{int_ex_OR_all_cell_types}_rank_50_{vargenes}.usages.k_{dim}.dt_2_0.consensus.txt"), header = TRUE, sep = "\t", row.names=1)
    # subset metadata time to match the embeddings
    final_meta = meta[match(rownames(embeddings), rownames(meta)),]

    # compute adjusted r squared
    r_sq = summary(lm(scale(final_meta$time) ~ scale(embeddings[,1:dim(embeddings)[2]])) )$adj.r.sq

    df = data.frame(method = glue("cNMF_{int_ex_OR_all_cell_types}"),r.adj=r_sq,dim=dim,vargenes=vargenes,Method="cNMF",CT=ct)

    final_df = rbind(final_df, df)
    return(final_df)

}

for (int_ex_OR_all_cell_types in c("int_ex", "all_cell_types") ){
    for (vargenes in c("3000","20000") ){
        #for (dim in c("5", "10", "15", "20", "25", "30", "35", "40", "45") ){
        for (dim in c("2", "3", "4", "6", "7", "8", "9") ){
            if (int_ex_OR_all_cell_types == "int_ex"){
                final_df = compute_rsq(int_ex_OR_all_cell_types=int_ex_OR_all_cell_types, vargenes=vargenes,dim=dim, ct="Interneurons & Ex. Neurons")
            } else {
                final_df = compute_rsq(int_ex_OR_all_cell_types=int_ex_OR_all_cell_types, vargenes=vargenes,dim=dim, ct="All Cell Types")
            }
        }
    }
}

final_df


method,r.adj,dim,vargenes,Method,CT
<glue>,<dbl>,<chr>,<chr>,<chr>,<chr>
cNMF_int_ex,0.038636518,2,3000,cNMF,Interneurons & Ex. Neurons
cNMF_int_ex,0.044186357,3,3000,cNMF,Interneurons & Ex. Neurons
cNMF_int_ex,0.119193324,4,3000,cNMF,Interneurons & Ex. Neurons
cNMF_int_ex,0.119611067,6,3000,cNMF,Interneurons & Ex. Neurons
cNMF_int_ex,0.119220787,7,3000,cNMF,Interneurons & Ex. Neurons
cNMF_int_ex,0.113845814,8,3000,cNMF,Interneurons & Ex. Neurons
cNMF_int_ex,0.112874743,9,3000,cNMF,Interneurons & Ex. Neurons
cNMF_int_ex,0.052400001,2,20000,cNMF,Interneurons & Ex. Neurons
cNMF_int_ex,0.039511390,3,20000,cNMF,Interneurons & Ex. Neurons


In [9]:
# have to rewrite index of outputed file
write.table(final_df, file = "/Users/shayecarver/condPCA/Light_Hvratin/cNMF_cum_rsq.txt", sep = "\t", quote = FALSE, append = TRUE,header=FALSE)



Warning message in write.table(final_df, file = "/Users/shayecarver/condPCA/Light_Hvratin/cNMF_cum_rsq.txt", :
"appending column names to file"


In [3]:
# final_df = read.table("/Users/shayecarver/condPCA/Light_Hvratin/cNMF_cum_rsq.txt", header = TRUE, sep = "\t")


In [32]:
# extract 50 dimensions from output
output = read.table("/Users/shayecarver/condPCA/Light_Hvratin/output.txt", header = TRUE, sep = " ")
output = output[output$method == "cNMF" & output$seed == 1 & output$num_cells == "all" ,]
output <- output[!duplicated(output), ]
output = output[c("method","adj.rsq","dim","vargenes","data_param")]
colnames(output) <- c("method","r.adj","dim","vargenes","CT")
output[output$CT == "all_cell_types",]$CT <- "All Cell Types"
output[output$CT == "int_ex",]$CT <- "Interneurons & Ex. Neurons"
output$Method = output$method
output = output[c("method","r.adj","dim","vargenes","Method","CT")]
output[output$CT == "All Cell Types",]$method = "cNMF_all_cell_types"
output[output$CT == "Interneurons & Ex. Neurons",]$method = "cNMF_int_ex"
output

,method,r.adj,dim,vargenes,Method,CT
,<chr>,<dbl>,<int>,<int>,<chr>,<chr>
927,cNMF_all_cell_types,0.07009209,50,3000,cNMF,All Cell Types
928,cNMF_int_ex,0.29391700,50,3000,cNMF,Interneurons & Ex. Neurons
964,cNMF_int_ex,0.41043037,50,20000,cNMF,Interneurons & Ex. Neurons
965,cNMF_all_cell_types,0.23209561,50,20000,cNMF,All Cell Types


In [33]:
final_df = rbind(final_df, output)
final_df

,method,r.adj,dim,vargenes,Method,CT
,<glue>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,cNMF_int_ex,0.118081215,5,3000,cNMF,Interneurons & Ex. Neurons
2,cNMF_int_ex,0.113561741,10,3000,cNMF,Interneurons & Ex. Neurons
3,cNMF_int_ex,0.283212801,15,3000,cNMF,Interneurons & Ex. Neurons
4,cNMF_int_ex,0.281746773,20,3000,cNMF,Interneurons & Ex. Neurons
5,cNMF_int_ex,0.281241776,25,3000,cNMF,Interneurons & Ex. Neurons
6,cNMF_int_ex,0.266407339,30,3000,cNMF,Interneurons & Ex. Neurons
7,cNMF_int_ex,0.266960998,35,3000,cNMF,Interneurons & Ex. Neurons
8,cNMF_int_ex,0.288100801,40,3000,cNMF,Interneurons & Ex. Neurons
9,cNMF_int_ex,0.295734397,45,3000,cNMF,Interneurons & Ex. Neurons


In [27]:
write.table(final_df, file = "/Users/shayecarver/condPCA/Light_Hvratin/cNMF_cum_rsq.txt", sep = "\t", quote = FALSE)
